In [2]:
import streamlit as st
# To make things easier later, we're also importing numpy and pandas for
# working with sample data.
import numpy as np
from numpy.random import default_rng
import pandas as pd
import make_standings
from make_standings import Standings
from itertools import permutations
import altair as alt
import time

In [4]:
rng = np.random.default_rng()

In [5]:
reps = 20
df = pd.DataFrame({"Team":list(reps*"abcde"),"Proportion":rng.random(reps*5),"Seed":np.random.randint(4,size=reps*5)})

In [6]:
df

,Team,Proportion,Seed
0,a,0.654374,3
1,b,0.976178,0
2,c,0.073840,2
3,d,0.054255,2
4,e,0.936080,3
...,...,...,...
95,a,0.938008,1
96,b,0.386415,0
97,c,0.552991,1
98,d,0.041393,0


In [7]:
alt.Chart(df).mark_bar().encode(
    x = "Team",
    y = "Proportion",
    color=alt.Color("Seed:O",scale=alt.Scale(scheme="Tableau10"),sort=alt.EncodingSortField('Seed', op='min')),
    tooltip=["Team","Proportion","Seed"],
    order=alt.Order("Seed",sort="ascending")
)

alt.Chart(...)

In [8]:
alt.Chart(df).mark_bar().encode(
    x = "Team",
    y = "Proportion",
    color=alt.Color("Seed:O",scale = alt.Scale(domain=[3,2,1,0],range=["blue","red","green","lightgrey"])),
    tooltip=["Team","Proportion","Seed"],
    order=alt.Order("Seed",sort="descending")
)

alt.Chart(...)

In [9]:
df = pd.read_csv("schedules/schedule21.csv")
pr_both = pd.read_csv("data/pr_both.csv",squeeze=True,index_col="Side")
div_series = pd.read_csv("data/divisions.csv",squeeze=True,index_col=0)
teams = sorted(list(set(df.team_home)))
conf_teams = {}
for conf in ["AFC","NFC"]:
    conf_teams[conf] = [t for t in teams if div_series[t][:3]==conf]
rng = np.random.default_rng()

In [10]:
def simulate_reg_season(df):
    df.loc[:,["score_home","score_away"]] = rng.normal(df[["mean_home","mean_away"]],10)
    df.loc[:,["score_home","score_away"]] = df.loc[:,["score_home","score_away"]].astype(int)
    adjust_ties(df)
    return None

def adjust_ties(df):
    tied_games = df.loc[df.score_home == df.score_away,["score_home","score_away"]].copy()
    x = rng.normal(size=len(tied_games))
    tied_games.iloc[np.where((x > -1.6) & (x < 0))[0]] += (0,3)
    tied_games.iloc[np.where(x >= 0)[0]] += (3,0)
    df.loc[tied_games.index,["score_home","score_away"]] = tied_games
    return None

In [11]:
df["pr_home_Off"] = df.team_home.map(lambda s: pr_both[s+"_Off"])
df["pr_home_Def"] = df.team_home.map(lambda s: pr_both[s+"_Def"])
df["pr_away_Off"] = df.team_away.map(lambda s: pr_both[s+"_Off"])
df["pr_away_Def"] = df.team_away.map(lambda s: pr_both[s+"_Def"])
df["mean_home"] = df["pr_home_Off"]-df["pr_away_Def"]+pr_both["mean_score"]+pr_both["HFA"]/2
df["mean_away"] = df["pr_away_Off"]-df["pr_home_Def"]+pr_both["mean_score"]-pr_both["HFA"]/2

In [12]:
total_dict = {}

win_dict = {t:{i:0 for i in range(18)} for t in teams}
for conf in ["AFC","NFC"]:
    total_dict[conf] = {i:{t:0 for t in sorted(conf_teams[conf])} for i in range(1,8)}

In [13]:
div_dict = {}

s = div_series
for a,b in s.groupby(s):
    div_dict[a] = list(b.index)

rank_dict = {div:{} for div in div_dict.keys()}

for div in rank_dict.keys():
    for team_sort in permutations(div_dict[div]):
        rank_dict[div][team_sort] = 0

In [14]:
reps = 30

start = time.time()

for i in range(reps):
    simulate_reg_season(df)
    stand = Standings(df)
    s = stand.standings
    s_dict = dict(zip(s.index,s.Wins))
    for t,w in s_dict.items():
        win_dict[t][w] += 1
    for d in rank_dict.keys():
        rank_dict[d][tuple(stand.div_ranks[d])] += 1
    p = stand.playoffs
    for conf in ["AFC","NFC"]:
        for j,t in enumerate(p[conf]):
            total_dict[conf][j+1][t] += 1

end = time.time()

In [15]:
stand.div_ranks

{'AFC East': ['MIA', 'BUF', 'NE', 'NYJ'],
 'AFC North': ['BAL', 'PIT', 'CLE', 'CIN'],
 'AFC South': ['JAX', 'TEN', 'IND', 'HOU'],
 'AFC West': ['KC', 'LAC', 'DEN', 'LVR'],
 'NFC East': ['DAL', 'PHI', 'NYG', 'WAS'],
 'NFC North': ['MIN', 'GB', 'CHI', 'DET'],
 'NFC South': ['NO', 'ATL', 'TB', 'CAR'],
 'NFC West': ['SEA', 'LAR', 'ARI', 'SF']}

In [16]:

            
for div in stand.div_ranks.keys():
            rank_dict[div][tuple(stand.div_ranks[div])] += 1

In [17]:
rank_dict

{'AFC East': {('BUF', 'MIA', 'NE', 'NYJ'): 3,
  ('BUF', 'MIA', 'NYJ', 'NE'): 3,
  ('BUF', 'NE', 'MIA', 'NYJ'): 8,
  ('BUF', 'NE', 'NYJ', 'MIA'): 3,
  ('BUF', 'NYJ', 'MIA', 'NE'): 1,
  ('BUF', 'NYJ', 'NE', 'MIA'): 2,
  ('MIA', 'BUF', 'NE', 'NYJ'): 2,
  ('MIA', 'BUF', 'NYJ', 'NE'): 1,
  ('MIA', 'NE', 'BUF', 'NYJ'): 2,
  ('MIA', 'NE', 'NYJ', 'BUF'): 0,
  ('MIA', 'NYJ', 'BUF', 'NE'): 0,
  ('MIA', 'NYJ', 'NE', 'BUF'): 0,
  ('NE', 'BUF', 'MIA', 'NYJ'): 3,
  ('NE', 'BUF', 'NYJ', 'MIA'): 1,
  ('NE', 'MIA', 'BUF', 'NYJ'): 1,
  ('NE', 'MIA', 'NYJ', 'BUF'): 0,
  ('NE', 'NYJ', 'BUF', 'MIA'): 0,
  ('NE', 'NYJ', 'MIA', 'BUF'): 0,
  ('NYJ', 'BUF', 'MIA', 'NE'): 1,
  ('NYJ', 'BUF', 'NE', 'MIA'): 0,
  ('NYJ', 'MIA', 'BUF', 'NE'): 0,
  ('NYJ', 'MIA', 'NE', 'BUF'): 0,
  ('NYJ', 'NE', 'BUF', 'MIA'): 0,
  ('NYJ', 'NE', 'MIA', 'BUF'): 0},
 'AFC North': {('BAL', 'CIN', 'CLE', 'PIT'): 4,
  ('BAL', 'CIN', 'PIT', 'CLE'): 0,
  ('BAL', 'CLE', 'CIN', 'PIT'): 3,
  ('BAL', 'CLE', 'PIT', 'CIN'): 8,
  ('BAL', 'PIT', '

In [20]:
def prob_to_odds(p):
    if p < .000001:
        return "NA"
    if p > 0.5:
        x = 100*p/(p-1)
        return f"{x:.0f}"
    elif p <= 0.5:
        x = 100*(1-p)/p
        return f"+{x:.0f}"

In [21]:
prob_to_odds(0)

'NA'

In [22]:
color_dict = {"blue": "#5778a4", "orange": "#e49444", "red": "#d1615d", "teal": "#85b6b2", "green": "#6a9f58", "yellow": "#e7ca60", "purple": "#a87c9f",}

chart_dict = {}

for conf in ["AFC","NFC"]:

    playoff_dicts = total_dict[conf]

    source = pd.DataFrame([(k,t,playoff_dicts[k][t]/reps) for k in playoff_dicts.keys() for t in conf_teams[conf]],
        columns = ["Seed","Team","Proportion"])

    for a,b in source.groupby("Team"):
        source.loc[source["Team"] == a, "Make playoffs"] = b.Proportion.sum()
        source.loc[source["Team"] == a,"Equal or better"] = b.Proportion.cumsum()

    #source["Equal or better"] += source["Proportion"]

    ordering = sorted(conf_teams[conf],key=lambda t: sum([playoff_dicts[i][t] for i in playoff_dicts.keys()]),reverse=True)
    ordering_seeds = list(range(7,0,-1))


    c = alt.Chart(source).mark_bar().encode(
        x=alt.X('Team',sort = ordering),
        y=alt.Y('Proportion',scale=alt.Scale(domain=[0,1]),sort=ordering_seeds),
        tooltip = [alt.Tooltip('Team'),alt.Tooltip('Seed', format=".0f"), alt.Tooltip('Proportion', format=".3f"),
                            alt.Tooltip('Equal or better', format=".3f"),alt.Tooltip('Make playoffs', format=".3f")],
        color=alt.Color('Seed:N', scale=alt.Scale(domain=[7,6,5,4,3,2,1],range=[color_dict[color] for color in ["purple","yellow","green","teal","red","orange","blue"]])),
        order=alt.Order(
                'Seed:N',
                sort='ascending'
            )
    ).properties(
        title=f"{conf} playoff seedings",
        width=500,
        height=500,
    )

    chart_dict[conf] = c

playoff_charts = alt.hconcat(*chart_dict.values()).resolve_scale(
    color='independent'
).properties(
    title=f"Based on {reps} simulations:"
)

In [23]:
odds_dict = {"Proportion":"Odds", "Make playoffs": "Odds to make playoffs", "Equal or better":"Odds of equal or better"}

In [24]:
for c in odds_dict.keys():
    source[odds_dict[c]] = source[c].map(prob_to_odds)

In [25]:
alt.Chart(source).mark_point().encode(
    x = "Proportion",
    y = "Make playoffs",
    tooltip = [alt.Tooltip("x:N",title="chris"),"y:N"],
).transform_calculate(
    x="format(datum.Proportion, ',.3f')+' (' +datum.Odds+')'",
)

alt.Chart(...)

In [26]:
source["Odds"]

0      +2900
1         NA
2         NA
3         NA
4       +900
       ...  
107     +900
108     +900
109     +500
110    +1400
111    +2900
Name: Odds, Length: 112, dtype: object

In [27]:
playoff_charts

alt.HConcatChart(...)

In [28]:
game_list = ["Mette", "Damir", "Amra", "Marie", "Chris"]

In [29]:
import numpy as np
rng = np.random.default_rng()
rng.permutation(game_list)

array(['Chris', 'Amra', 'Mette', 'Marie', 'Damir'], dtype='<U5')

In [30]:
series = df.apply(lambda row: rng.permutation(game_list), axis=1)

In [31]:
df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,pr_spread,pr_total,pr_hfa,in_div,in_conf,pr_home_Off,pr_home_Def,pr_away_Off,pr_away_Def,mean_home,mean_away
0,9/9/2021,2021,1,TB,28,33,DAL,NaN,NaN,NaN,False,True,4.250773,1.214125,1.441763,-0.518368,29.644140,22.992638
1,9/12/2021,2021,1,ATL,36,10,PHI,NaN,NaN,NaN,False,True,-0.043689,-1.642589,-1.920606,-1.421908,26.253218,22.486983
2,9/12/2021,2021,1,BUF,24,22,PIT,NaN,NaN,NaN,False,True,3.371342,1.116772,-0.146587,0.888235,27.358107,21.501640
3,9/12/2021,2021,1,CAR,21,19,NYJ,NaN,NaN,NaN,False,False,-1.474896,-0.923250,-3.261434,-0.929074,24.329178,20.426817
4,9/12/2021,2021,1,CIN,36,26,MIN,NaN,NaN,NaN,False,False,-1.832549,-1.644577,0.058605,0.203009,22.839442,24.468182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,1/9/2021,2021,18,TB,24,26,CAR,NaN,NaN,NaN,True,True,4.250773,1.214125,-1.474896,-0.923250,30.049023,20.075978
268,1/9/2021,2021,18,ARI,22,25,SEA,NaN,NaN,NaN,True,True,1.262909,-0.315893,1.118272,0.485775,25.652134,24.199165
269,1/9/2021,2021,18,DEN,11,10,KC,NaN,NaN,NaN,True,True,-1.677494,1.160501,5.354772,1.327564,21.869942,26.959271
270,1/9/2021,2021,18,LVR,21,35,LAC,NaN,NaN,NaN,True,True,0.087404,-1.708777,0.429096,-0.028791,24.991195,24.902873


In [32]:
def highlight_cols(s):
    color = 'yellow'
    return 'background-color: %s' % color

In [60]:
df = pd.DataFrame(index=range(10**2),columns = range(5))
for i in range(len(df)):
    df.loc[i] = rng.permutation(game_list)
df = df.style.applymap(highlight_cols, subset=pd.IndexSlice[:, 0:2])

In [61]:
df

,0,1,2,3,4
0,Chris,Amra,Mette,Marie,Damir
1,Chris,Amra,Marie,Mette,Damir
2,Chris,Mette,Amra,Damir,Marie
3,Amra,Mette,Damir,Marie,Chris
4,Damir,Marie,Amra,Chris,Mette
5,Damir,Amra,Mette,Chris,Marie
6,Damir,Marie,Chris,Mette,Amra
7,Chris,Damir,Mette,Marie,Amra
8,Mette,Marie,Damir,Amra,Chris
9,Amra,Chris,Marie,Damir,Mette


In [62]:
df.data = df.data.iloc[2:10]

In [63]:
df

,0,1,2,3,4
2,Chris,Mette,Amra,Damir,Marie
3,Amra,Mette,Damir,Marie,Chris
4,Damir,Marie,Amra,Chris,Mette
5,Damir,Amra,Mette,Chris,Marie
6,Damir,Marie,Chris,Mette,Amra
7,Chris,Damir,Mette,Marie,Amra
8,Mette,Marie,Damir,Amra,Chris
9,Amra,Chris,Marie,Damir,Mette
